###Packages

In [ ]:
!pip install SPARQLWrapper
!pip install openai
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


###getting correct answers from endpoint Booleans not working check 2024-02-02 revisions

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
from tqdm import trange
dbpedia_endpoint = "http://dbpedia.org/sparql"


def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        #error_count+=1
        #print(error_count)
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

queries = []
with open('/content/LCQUAD-train-data.json', 'r') as f:
  data = json.load(f)
  queries = [d['sparql_query'] for d in data]
  questions = [d['corrected_question'] for d in data]
  ids = [d['sparql_template_id'] for d in data]
  final = []
  #gold_ans = [d['new_answers'] for d in data]
  #pred = [d['pred'] for d in data]
  #answers = [d['answers'] for d in data]
  #triples = [d['triples'] for d in data]
  #new_triples = [d['new_triples'] for d in data]
  #new_query = [d['new_query'] for d in data]
  #print(ids)
  # for j in range(len(gold_ans)):
  #     if len(list(gold_ans[j].keys())) == 0:
  #         ans = []
  #     else:
  #         assert len(list(gold_ans[j].keys())) == 1
  #         gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))
final = []
#print(type(final))
# print(len(new_triples))
# print(len(queries))
gold_answers = {}
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    #print(results)
    try:
      for result in results["results"]["bindings"]:
          for var_name, var_data in result.items():
            var_value = var_data["value"]
            #print(var_name)
            #print(var_value)
            if var_name in pred_answers.keys():
              pred_answers[var_name].append(var_value)
            else:
              pred_answers[var_name] = [var_value]
      gold_answers[i]=pred_answers
    except:
      print('probably a boolean')
    if 'boolean' in results.keys():
      gold_answers[i] = results['boolean']
    #print(gold_answers[i])
      #for result in results['results']['bindings']:
        #print(result)
    #print("the pred answer is :", pred_answers)
    #print(gold_ans[i])
    final.append({'id': ids[i],'query': query, 'question':questions[i], 'answers':gold_answers[i]})

hybrid = json.dumps(final, indent=4)
with open('/content/test4.json', 'w') as outfile:
    outfile.write(hybrid)




FileNotFoundError: [Errno 2] No such file or directory: '/content/LCQUAD-train-data.json'

###compuing hits @1 or micro f1@1 I think TN is acctually considered a TP here


In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
from tqdm import trange
dbpedia_endpoint = "http://dbpedia.org/sparql"


error_count = 0
def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        #error_count+=1
        #print(error_count)
        return None

def process_text(txt):
    #return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')
    return txt

##use hits @1 instead
def eval(gold, pred, boolean=False):
  TP, FP, FN = 0, 0, 0
  print(gold, pred)
  # if gold is None and pred is None:
  #       return 0, 0, 0  # Return default values if both are None

  # if gold is None and not pred is None:
  #       return 0, 1, 0  # Return default values if either is None

  # if gold is not None and pred is None:
  #       return 0, 0, 1  # Return default values if either is None

  # if gold is None or pred is None:
  #       return 0, 0, 0  # Return default values if either is None
  print(type(pred))
  print(type(gold))
  for item in pred:
    print('we started the loop')
    new_item = process_text(item) if not boolean else item
    print('we processed the text')

    #print(new_item)
    if new_item in gold:
      #print('checked if new item in gold')
      TP += 1
      #print('added to TP')
      #return TP, FP, FN
    else:
      FP += 1
      print('added to FP')
      #return TP, FP, FN
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    if item and not pred:
      #print(item, pred)
      FN += 1
    return TP, FP, FN

# def eval(gold, pred, boolean=False):
#     TP, FP, FN = 0, 0, 0
#     for item in pred:
#         new_item = process_text(item) if not boolean else item
#         print(new_item)
#         if new_item in gold:
#             TP += 1
#         else:
#             FP += 1
#             #print("we had a flase positive")
#     pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
#     for item in gold:
#         if item not in pred:
#             FN += 1
#             #print('we had a flase negative')
#     acc = TP / len(pred)
#     recall = TP / (TP + FN)
#     precision = TP / (TP + FP)
#     f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
#     if f1 !=1:
#       print("this one did not work")
#     rec, pre = recall, precision
#     return acc, rec, pre, f1

queries = []
TN_t=0
TP_t=0
FP_t=0
FN_t=0
with open('/content/mistral-7b-ER-5_epochs_test_parsed.json', 'r') as f:
  data = json.load(f)
  queries = [d['predicted_query'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['new_answers'] for d in data]
  for j in range(len(gold_ans)):
      #print(type(gold_ans[j]))
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

#total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    #print(query)
    pred_answers = {}
    results = execute_sparql_query(query)

    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        #error_count+=1
        TP_t+=1
        # total_acc += 1
        # total_rec += 1
        # total_pre += 1
        # total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1

        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1

        continue
    if 'boolean' in results.keys():
        TP, FP, FN = eval([gold_ans[i]], [results['boolean']], boolean=True)
        TP_t+=TP
        FP_t+=FP
        FN_t+=FN
        print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)
        # total_acc += acc
        # total_rec += rec
        # total_pre += pre
        # total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        # total_acc += 1
        # total_rec += 1
        # total_pre += 1
        # total_f1 += 1
        TN_t+=1
        print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)

        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        FN_t+=1
        print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)

        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        FP_t+=1
        print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)

        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        FP_t+=1
        print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)

        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)

        continue
    pred_ans = next(iter(pred_answers.values()))
    #print(f"gold_ans[{i}]: {gold_ans[i]}")
    TP, FP, FN = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    TP_t+=TP
    FP_t+=FP
    FN_t+=FN
    print(TP_t)
    print(TN_t)
    print(FP_t)
    print(FN_t)
    print(error_count)
    print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)
    # total_acc += acc
    # total_rec += rec
    # total_pre += pre
    # total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
# print(f'\nacc: {total_acc / len(queries)}')
# print(f'rec: {total_rec / len(queries)}')
# print(f'pre: {total_pre / len(queries)}')
# print(f'f1: {total_f1 / len(queries)}')
print(TP_t)
print(TN_t)
print(FP_t)
print(FN_t)
print(error_count)
print("total so far is", TP_t+TN_t+FP_t+FN_t+error_count)
TP_t=TP_t+TN_t
prec=TP_t / (TP_t+FP_t)
rec =TP_t / (TP_t+FN_t+error_count)

f1 = ((2 * (prec * rec)) / (prec + rec))
print(f'rec: {rec}')
print(f'pre: {prec}')
print(f'f1: {f1}')

FileNotFoundError: [Errno 2] No such file or directory: '/content/mistral-7b-ER-5_epochs_test_parsed.json'

###compute Mahdiyar f1 Macro F1

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
from tqdm import trange
dbpedia_endpoint = "http://dbpedia.org/sparql"


error_count = 0
def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        #error_count+=1
        #print(error_count)
        return None

def process_text(txt):
    #return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')
    return txt
def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/mistral-7b-5-RT-5_epochs_test_parsed.json', 'r', encoding='utf-8' ) as f:
  data = json.load(f)
  queries = [d['predicted_query'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['new_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        #error_count+=1
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1

        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1

        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    #print(pred_ans, gold_ans[i])
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')
print('errors: ', error_count)

  1%|▏         | 2/150 [00:01<01:51,  1.33it/s]


empty predicted answer set for sample #98


  2%|▏         | 3/150 [00:02<01:53,  1.30it/s]


empty predicted answer set for sample #86


  3%|▎         | 4/150 [00:03<01:47,  1.35it/s]

...


empty predicted answer set for sample #14


 99%|█████████▊| 148/150 [01:45<00:01,  1.40it/s]


empty predicted answer set for sample #148


100%|██████████| 150/150 [01:46<00:00,  1.41it/s]


empty predicted answer set for sample #179

acc: 0.42933139241179447
rec: 0.44193882043943084
pre: 0.42933139241179447
f1: 0.4247702033614239
errors:  6


**Compute** Macro F1 QALD metric from https://ceur-ws.org/Vol-2241/paper-06.pdf
Same as regular Macro F1 but If the golden answerset is not empty but the QA system responds with an empty answerset, it is assumed that the system determined that it cannot answer the question. Here we set the precision to 1 and the recall and Fmeasure to 0.


In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
from tqdm import trange
dbpedia_endpoint = "http://dbpedia.org/sparql"


error_count = 0
def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        #error_count+=1
        #print(error_count)
        return None

def process_text(txt):
    #return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')
    return txt

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/mistral_instruct_v2_no_triples_7epochs_LCQUAD_20_bm25_triples_test.json', 'r') as f:
  data = json.load(f)
  queries = [d['predicted_query'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1
        # total_acc += 1
        # total_rec += 1
        # total_pre += 1
        # total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1
        total_pre+=1

        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        error_count+=1

        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    print(pred_ans, gold_ans[i])
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')
print(error_count)

  0%|          | 2/1000 [00:01<14:04,  1.18it/s]


empty predicted answer set for sample #3293


  0%|          | 4/1000 [00:03<14:15,  1.16it/s]

['http://dbpedia.org/resource/Chess'] ['http://dbpedia.org/resource/Chess']


  1%|          | 7/1000 [00:05<13:03,  1.27it/s]

...

 24%|██▍       | 243/1000 [03:20<14:11,  1.12s/it]


empty predicted answer set for sample #1141


 25%|██▍       | 246/1000 [03:22<10:22,  1.21it/s]


empty predicted answer set for sample #2551


KeyboardInterrupt: 

#### T5-3B w/ 10 triples using ChatGPT including second response
model_2: t5-3b-tri-10-ep-10-t-v2

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_t5-3b-tri-10-ep-10-t_qald-9-plus_v2.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  queries_2 = [d['pred_sparql_2'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    query_2 = queries_2[i]
    pred_answers = {}
    pred_answers_2 = {}
    results = execute_sparql_query(query)
    results_2 = execute_sparql_query(query_2)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results) and (not results_2): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results) and (not results_2):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    use_pred_ans_2 = None
    if (not results) and (results_2):
        use_pred_ans_2 = True
    elif (results) and (not results_2):
        use_pred_ans_2 = False
    if (use_pred_ans_2 != True) and 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        acc_2, rec_2, pre_2, f1_2 = 0, 0, 0, 0
        if (use_pred_ans_2 != False) and 'boolean' in results_2.keys():
            acc_2, rec_2, pre_2, f1_2 = eval([gold_ans[i]], [results_2['boolean']], boolean=True)
        total_acc += max(acc, acc_2)
        total_rec += max(rec, rec_2)
        total_pre += max(pre, pre_2)
        total_f1 += max(f1, f1_2)
        continue
    elif (use_pred_ans_2 != False) and 'boolean' in results_2.keys():
        acc_2, rec_2, pre_2, f1_2 = eval([gold_ans[i]], [results_2['boolean']], boolean=True)
        total_acc += acc_2
        total_rec += rec_2
        total_pre += pre_2
        total_f1 += f1_2
        continue
    else:
        if (use_pred_ans_2 != True):
            for result in results["results"]["bindings"]:
                for var_name, var_data in result.items():
                    var_value = var_data["value"]
                    if var_name in pred_answers.keys():
                        pred_answers[var_name].append(var_value)
                    else:
                        pred_answers[var_name] = [var_value]
        if (use_pred_ans_2 != False):
            for result in results_2["results"]["bindings"]:
                for var_name, var_data in result.items():
                    var_value = var_data["value"]
                    if var_name in pred_answers_2.keys():
                        pred_answers_2[var_name].append(var_value)
                    else:
                        pred_answers_2[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0 or len(list(pred_answers_2.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0) and (len(list(pred_answers_2.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0 and len(list(pred_answers_2.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        if (use_pred_ans_2 != True):
            assert len(list(pred_answers.keys())) == 1
    except:
        try:
            if (use_pred_ans_2 != False):
                assert len(list(pred_answers_2.keys())) == 1
            use_pred_ans_2 = True
        except:
            print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
            continue
    pred_ans = next(iter(pred_answers.values())) if (not use_pred_ans_2) else next(iter(pred_answers_2.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    acc_2, rec_2, pre_2, f1_2 = 0, 0, 0, 0
    if not use_pred_ans_2 and len(list(pred_answers_2.keys())) > 0:
        pred_ans_2 = next(iter(pred_answers_2.values()))
        acc_2, rec_2, pre_2, f1_2 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans_2)
    total_acc += max(acc, acc_2)
    total_rec += max(rec, rec_2)
    total_pre += max(pre, pre_2)
    total_f1 += max(f1, f1_2)
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  3%|▎         | 4/150 [00:05<03:25,  1.41s/it]


empty predicted answer set for sample #3


  8%|▊         | 12/150 [00:20<03:51,  1.68s/it]


empty predicted answer set for sample #11


  9%|▊         | 13/150 [00:22<03:37,  1.59s/it]

...


empty predicted answer set for sample #147


 99%|█████████▉| 149/150 [03:40<00:01,  1.45s/it]


empty predicted answer set for sample #148


100%|██████████| 150/150 [03:41<00:00,  1.48s/it]


acc: 0.5515402324779858
rec: 0.5881299209194957
pre: 0.5515402324779858
f1: 0.5504456538309445


#### T5-3B w/ 10 triples only for training. ChatGPT is used to correct the output
model_3: t5-3b-tri-10-ep-10-t-v3

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_t5-3b-tri-10-ep-10-t_qald-9-plus_v3.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|          | 1/150 [00:00<02:03,  1.21it/s]


empty predicted answer set for sample #0


  1%|▏         | 2/150 [00:01<01:42,  1.44it/s]


empty predicted answer set for sample #1


  3%|▎         | 5/150 [00:03<01:30,  1.59it/s]

...


empty predicted answer set for sample #146


 99%|█████████▉| 149/150 [03:28<00:00,  1.58it/s]


empty predicted answer set for sample #148


100%|██████████| 150/150 [03:29<00:00,  1.39s/it]


acc: 0.14666666666666667
rec: 0.14666666666666667
pre: 0.14666666666666667
f1: 0.14666666666666667


#### ChatGPT zero-shot setting
model_4: gpt-3.5-turbo-v1

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|▏         | 2/150 [00:01<02:02,  1.21it/s]


empty predicted answer set for sample #1


  2%|▏         | 3/150 [00:02<01:50,  1.32it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'wdt:P31\' before \'wd:Q8502\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?mountain ?mountainLabel\nWHERE {\n  ?mountain wdt:P31 wd:Q8502;  # instance of mountain\n           wdt:P17 wd:Q183;    # located in Germany\n           wdt:P610 ?elevation. # has elevation\n  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }\n}\nORDER BY DESC(?elevation)\nLIMIT 1\n'

syntax error for sample #2


  3%|▎         | 4/150 [00:03<01:45,  1.39it/s]

...


empty predicted answer set for sample #146


 99%|█████████▊| 148/150 [01:41<00:01,  1.48it/s]


empty predicted answer set for sample #147


100%|██████████| 150/150 [01:43<00:00,  1.45it/s]


empty predicted answer set for sample #149

acc: 0.31408982073952024
rec: 0.32679280666365346
pre: 0.31408982073952024
f1: 0.30793380900584283


#### ChatGPT zero-shot setting including second response
model_5: gpt-3.5-turbo-v2

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  queries_2 = [d['pred_sparql_2'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    query_2 = queries_2[i]
    pred_answers = {}
    pred_answers_2 = {}
    results = execute_sparql_query(query)
    results_2 = execute_sparql_query(query_2)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results) and (not results_2): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results) and (not results_2):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    use_pred_ans_2 = None
    if (not results) and (results_2):
        use_pred_ans_2 = True
    elif (results) and (not results_2):
        use_pred_ans_2 = False
    if (use_pred_ans_2 != True) and 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        acc_2, rec_2, pre_2, f1_2 = 0, 0, 0, 0
        if (use_pred_ans_2 != False) and 'boolean' in results_2.keys():
            acc_2, rec_2, pre_2, f1_2 = eval([gold_ans[i]], [results_2['boolean']], boolean=True)
        total_acc += max(acc, acc_2)
        total_rec += max(rec, rec_2)
        total_pre += max(pre, pre_2)
        total_f1 += max(f1, f1_2)
        continue
    elif (use_pred_ans_2 != False) and 'boolean' in results_2.keys():
        acc_2, rec_2, pre_2, f1_2 = eval([gold_ans[i]], [results_2['boolean']], boolean=True)
        total_acc += acc_2
        total_rec += rec_2
        total_pre += pre_2
        total_f1 += f1_2
        continue
    else:
        if (use_pred_ans_2 != True):
            for result in results["results"]["bindings"]:
                for var_name, var_data in result.items():
                    var_value = var_data["value"]
                    if var_name in pred_answers.keys():
                        pred_answers[var_name].append(var_value)
                    else:
                        pred_answers[var_name] = [var_value]
        if (use_pred_ans_2 != False):
            for result in results_2["results"]["bindings"]:
                for var_name, var_data in result.items():
                    var_value = var_data["value"]
                    if var_name in pred_answers_2.keys():
                        pred_answers_2[var_name].append(var_value)
                    else:
                        pred_answers_2[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0 or len(list(pred_answers_2.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0) and (len(list(pred_answers_2.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0 and len(list(pred_answers_2.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        if (use_pred_ans_2 != True):
            assert len(list(pred_answers.keys())) == 1
    except:
        try:
            if (use_pred_ans_2 != False):
                assert len(list(pred_answers_2.keys())) == 1
            use_pred_ans_2 = True
        except:
            print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
            continue
    pred_ans = next(iter(pred_answers.values())) if (not use_pred_ans_2) else next(iter(pred_answers_2.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    acc_2, rec_2, pre_2, f1_2 = 0, 0, 0, 0
    if not use_pred_ans_2 and len(list(pred_answers_2.keys())) > 0:
        pred_ans_2 = next(iter(pred_answers_2.values()))
        acc_2, rec_2, pre_2, f1_2 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans_2)
    total_acc += max(acc, acc_2)
    total_rec += max(rec, rec_2)
    total_pre += max(pre, pre_2)
    total_f1 += max(f1, f1_2)
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|▏         | 2/150 [00:02<03:28,  1.41s/it]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at 'dbpedia-owl:Person' before '.'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?killer WHERE {\n  ?killer dbp:perpetratorOf <http://dbpedia.org/resource/Assassination_of_Julius_Caesar>.\n  ?killer rdf:type dbpedia-owl:Person.\n}\nLIMIT 100\n"

empty predicted answer set for sample #1

error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'wdt:P31\' before \'wd:Q8502\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?mountain ?mountainLabel\nWHERE {\n  ?mountain wdt:P31 wd:Q850

  2%|▏         | 3/150 [00:04<03:23,  1.39s/it]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'wdt:P31\' before \'wd:Q8502\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?mountain ?mountainLabel\nWHERE {\n  ?mountain wdt:P31 wd:Q8502;  # instance of mountain\n           wdt:P17 wd:Q183;    # located in Germany\n           wdt:P610 ?elevation. # has elevation\n  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }\n}\nORDER BY DESC(?elevation)\nLIMIT 1\n'

syntax error for sample #2


  3%|▎         | 4/150 [00:05<03:19,  1.37s/it]

...


results dict has more keys than expected for sample #146


 99%|█████████▊| 148/150 [03:24<00:02,  1.36s/it]


empty predicted answer set for sample #147


100%|██████████| 150/150 [03:27<00:00,  1.38s/it]


empty predicted answer set for sample #149

acc: 0.3352612064109059
rec: 0.3473701444947655
pre: 0.3352612064109059
f1: 0.32919990916986175


#### ChatGPT zero-shot setting w/ 10 triples
model_6: gpt-3.5-turbo-tri-10

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo-tri-10_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  3%|▎         | 4/150 [00:02<01:44,  1.39it/s]


empty predicted answer set for sample #3


  6%|▌         | 9/150 [00:07<02:45,  1.18s/it]


results are not of type boolean for sample #8


  7%|▋         | 10/150 [00:08<02:23,  1.02s/it]

...


empty predicted answer set for sample #143


 97%|█████████▋| 146/150 [02:16<00:02,  1.47it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 5: syntax error at '}' before '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?trail\nWHERE {\n  ?trail <http://dbpedia.org/property/hazards> ?hazard .\n  FILTER NOT EXISTS { ?hazard <http://dbpedia.org/resource/Flash_flood> }\n}\n"

syntax error for sample #145


100%|██████████| 150/150 [02:19<00:00,  1.08it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'dct:subject\' before \'dbc:Christopher_Columbus\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?shipName WHERE {\n  ?ship rdf:type dbo:Ship .\n  ?ship dct:subject dbc:Christopher_Columbus .\n  ?ship rdfs:label ?shipName .\n  FILTER(lang(?shipName) = "en")\n} LIMIT 3\n'

syntax error for sample #149

acc: 0.4499006992849857
rec: 0.4577389087006439
pre: 0.4499006992849857
f1: 0.4485660975070531


In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo-tri-10_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  3%|▎         | 4/150 [00:02<01:41,  1.43it/s]


empty predicted answer set for sample #3


  6%|▌         | 9/150 [00:07<02:48,  1.20s/it]


results are not of type boolean for sample #8


  7%|▋         | 10/150 [00:08<02:25,  1.04s/it]

...


empty predicted answer set for sample #143


 97%|█████████▋| 146/150 [02:15<00:02,  1.48it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 5: syntax error at '}' before '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?trail\nWHERE {\n  ?trail <http://dbpedia.org/property/hazards> ?hazard .\n  FILTER NOT EXISTS { ?hazard <http://dbpedia.org/resource/Flash_flood> }\n}\n"

syntax error for sample #145


100%|██████████| 150/150 [02:18<00:00,  1.08it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'dct:subject\' before \'dbc:Christopher_Columbus\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?shipName WHERE {\n  ?ship rdf:type dbo:Ship .\n  ?ship dct:subject dbc:Christopher_Columbus .\n  ?ship rdfs:label ?shipName .\n  FILTER(lang(?shipName) = "en")\n} LIMIT 3\n'

syntax error for sample #149

acc: 0.46079813518242163
rec: 0.4707944642561994
pre: 0.46079813518242163
f1: 0.4604137165546722


In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo-tri-10_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  3%|▎         | 4/150 [00:02<01:46,  1.38it/s]


empty predicted answer set for sample #3


  6%|▌         | 9/150 [00:08<02:58,  1.27s/it]


results are not of type boolean for sample #8


  7%|▋         | 10/150 [00:08<02:31,  1.08s/it]

...


empty predicted answer set for sample #143


 97%|█████████▋| 146/150 [01:57<00:02,  1.47it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 5: syntax error at '}' before '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?trail\nWHERE {\n  ?trail <http://dbpedia.org/property/hazards> ?hazard .\n  FILTER NOT EXISTS { ?hazard <http://dbpedia.org/resource/Flash_flood> }\n}\n"

syntax error for sample #145


100%|██████████| 150/150 [02:00<00:00,  1.24it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'dct:subject\' before \'dbc:Christopher_Columbus\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?shipName WHERE {\n  ?ship rdf:type dbo:Ship .\n  ?ship dct:subject dbc:Christopher_Columbus .\n  ?ship rdfs:label ?shipName .\n  FILTER(lang(?shipName) = "en")\n} LIMIT 3\n'

syntax error for sample #149

acc: 0.44156736595165247
rec: 0.4510722420339772
pre: 0.44156736595165247
f1: 0.4409470498880054


#### T5-3B zero-shot setting w/ 10 triples
model_7: t5-3b-tri-10-t

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_t5-3b-tri-10-t_qald-9-plus_v2.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|          | 1/150 [00:00<02:01,  1.22it/s]


error executing SPARQL query: HTTP Error 503: Service Temporarily Unavailable

syntax error for sample #0


  1%|▏         | 2/150 [00:01<01:49,  1.36it/s]


empty predicted answer set for sample #1


  3%|▎         | 4/150 [00:02<01:42,  1.42it/s]

...


error executing SPARQL query: HTTP Error 503: Service Temporarily Unavailable

syntax error for sample #147


 99%|█████████▉| 149/150 [02:17<00:00,  1.40it/s]


results dict has more keys than expected for sample #148


100%|██████████| 150/150 [02:18<00:00,  1.09it/s]


error executing SPARQL query: HTTP Error 503: Service Temporarily Unavailable

syntax error for sample #149

acc: 0.21840000000000004
rec: 0.2116381995917929
pre: 0.21840000000000004
f1: 0.21231134003760935


#### T5-3B zero-shot setting
model_8: t5-3b-t

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_t5-3b-t_qald-9-plus_v2.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|          | 1/150 [00:01<02:40,  1.07s/it]


results dict has more keys than expected for sample #0


  1%|▏         | 2/150 [00:01<02:18,  1.07it/s]


error executing SPARQL query: HTTP Error 503: Service Temporarily Unavailable

syntax error for sample #1


  2%|▏         | 3/150 [00:02<01:59,  1.23it/s]

...


results dict has more keys than expected for sample #147


 99%|█████████▉| 149/150 [02:04<00:00,  1.38it/s]


error executing SPARQL query: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error SR186:SECURITY: No permission to execute procedure DB.DBA.SPARUL_RUN with user ID 111, group ID 111\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n'

syntax error for sample #148


100%|██████████| 150/150 [02:04<00:00,  1.20it/s]


empty predicted answer set for sample #149

acc: 0.11173333333333332
rec: 0.10497153292512622
pre: 0.11173333333333332
f1: 0.10564467337094269


#### T5-3B using ChatGPT
model_9: t5-3b-ep-10-t-v1

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_t5-3b-ep-10-t_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|▏         | 2/150 [00:01<01:55,  1.28it/s]


empty predicted answer set for sample #1


  3%|▎         | 4/150 [00:03<02:04,  1.17it/s]


empty predicted answer set for sample #3


  4%|▍         | 6/150 [00:04<01:47,  1.34it/s]

...


empty predicted answer set for sample #146


 99%|█████████▊| 148/150 [01:47<00:01,  1.47it/s]


empty predicted answer set for sample #147


100%|██████████| 150/150 [01:50<00:00,  1.35it/s]


empty predicted answer set for sample #149

acc: 0.3020489932117839
rec: 0.31040091477176157
pre: 0.3020489932117839
f1: 0.2994036964583604


#### T5-3B w/ 10 triples only for test. To correct the syntax uses ChatGPT
model_10: t5-3b-ep-10-t-v2

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo_t5-3b-ep-10-t_qald-9-plus_v2.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|▏         | 2/150 [00:01<02:01,  1.22it/s]


empty predicted answer set for sample #1


  2%|▏         | 3/150 [00:02<01:51,  1.32it/s]


empty predicted answer set for sample #2


  3%|▎         | 4/150 [00:02<01:45,  1.38it/s]

...


empty predicted answer set for sample #146


 99%|█████████▊| 148/150 [01:50<00:01,  1.36it/s]


empty predicted answer set for sample #147


100%|██████████| 150/150 [01:51<00:00,  1.34it/s]


empty predicted answer set for sample #149

acc: 0.3132978837327292
rec: 0.32484806484391165
pre: 0.3132978837327292
f1: 0.30899389778060526


#### ChatGPT zero-shot setting w/ 20 triples
model_11: gpt-3.5-turbo-tri-20

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo-tri-20_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  3%|▎         | 4/150 [00:02<01:41,  1.44it/s]


empty predicted answer set for sample #3


  6%|▌         | 9/150 [00:07<02:53,  1.23s/it]


results are not of type boolean for sample #8


  7%|▋         | 10/150 [00:08<02:27,  1.06s/it]

...


empty predicted answer set for sample #143


 97%|█████████▋| 146/150 [02:15<00:02,  1.47it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 5: syntax error at '}' before '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?trail\nWHERE {\n  ?trail <http://dbpedia.org/property/hazards> ?hazard .\n  FILTER NOT EXISTS { ?hazard <http://dbpedia.org/resource/Flash_flood> }\n}\n"

syntax error for sample #145


100%|██████████| 150/150 [02:18<00:00,  1.08it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 4: Undefined namespace prefix in prefix:localpart notation at \'dct:subject\' before \'dbc:Christopher_Columbus\'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?shipName WHERE {\n  ?ship rdf:type dbo:Ship .\n  ?ship dct:subject dbc:Christopher_Columbus .\n  ?ship rdfs:label ?shipName .\n  FILTER(lang(?shipName) = "en")\n} LIMIT 3\n'

syntax error for sample #149

acc: 0.45418197751443473
rec: 0.46735068118741635
pre: 0.45418197751443473
f1: 0.45419785472070384


#### ChatGPT zero-shot setting w/ 5 triples
model_12: gpt-3.5-turbo-tri-5

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo-tri-5_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  3%|▎         | 4/150 [00:02<01:44,  1.39it/s]


empty predicted answer set for sample #3


  6%|▌         | 9/150 [00:06<01:35,  1.47it/s]


error executing SPARQL query: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error SR186:SECURITY: No permission to execute procedure DB.DBA.SPARUL_RUN with user ID 111, group ID 111\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n'

syntax error for sample #8


  7%|▋         | 10/150 [00:07<01:35,  1.47it/s]

...


results dict has more keys than expected for sample #142


 97%|█████████▋| 146/150 [02:15<00:02,  1.42it/s]


error executing SPARQL query: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 5: syntax error at '}' before '}'\n\nSPARQL query:\n#output-format:application/sparql-results+json\nSELECT ?trail\nWHERE {\n  ?trail <http://dbpedia.org/property/hazards> ?hazard .\n  FILTER NOT EXISTS { ?hazard <http://dbpedia.org/resource/Flash_flood> }\n}\n"

syntax error for sample #145


100%|██████████| 150/150 [02:18<00:00,  1.09it/s]


empty predicted answer set for sample #149

acc: 0.4658437655521407
rec: 0.4848358526696809
pre: 0.4658437655521407
f1: 0.46571285459016454


#### ChatGPT zero-shot setting w/ 1 triple
model_13: gpt-3.5-turbo-tri-1

In [ ]:
import requests
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from copy import deepcopy
dbpedia_endpoint = "http://dbpedia.org/sparql"

def execute_sparql_query(query):
    sparql = SPARQLWrapper(dbpedia_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.queryAndConvert()
        return results
    except Exception as e:
        print(f"\nerror executing SPARQL query: {e}")
        return None

def process_text(txt):
    return txt.strip().lower().replace(':','').replace('.','').replace('/','').replace('_','')

def eval(gold, pred, boolean=False):
    TP, FP, FN = 0, 0, 0
    for item in pred:
        new_item = process_text(item) if not boolean else item
        if new_item in gold:
            TP += 1
        else:
            FP += 1
    pred = list(map(lambda x: process_text(x), pred)) if not boolean else pred
    for item in gold:
        if item not in pred:
            FN += 1
    acc = TP / len(pred)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1 = ((2 * (precision * recall)) / (precision + recall)) if (precision + recall != 0) else 0
    rec, pre = recall, precision
    return acc, rec, pre, f1

queries = []
with open('/content/gpt-3.5-turbo-tri-1_qald-9-plus.json', 'r') as f:
  data = json.load(f)
  queries = [d['pred_sparql'] for d in data]
  ids = [d['id'] for d in data]
  gold_ans = [d['gold_answers'] for d in data]
  for j in range(len(gold_ans)):
      if len(list(gold_ans[j].keys())) == 0:
          ans = []
      else:
          assert len(list(gold_ans[j].keys())) == 1
          gold_ans[j] = deepcopy(next(iter(gold_ans[j].values())))

total_acc, total_rec, total_pre, total_f1 = 0, 0, 0, 0
for i in trange(len(queries)):
    query = queries[i]
    pred_answers = {}
    results = execute_sparql_query(query)
    if (type(gold_ans[i]) != bool and len(gold_ans[i]) == 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif type(gold_ans[i]) != bool and (len(gold_ans[i]) > 0) and (not results): # if results == None -> syntax error
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool and (not results):
        print(f'\nsyntax error for sample #{ids[i]}')
        continue
    if 'boolean' in results.keys():
        acc, rec, pre, f1 = eval([gold_ans[i]], [results['boolean']], boolean=True)
        total_acc += acc
        total_rec += rec
        total_pre += pre
        total_f1 += f1
        continue
    else:
        for result in results["results"]["bindings"]:
            for var_name, var_data in result.items():
                var_value = var_data["value"]
                if var_name in pred_answers.keys():
                    pred_answers[var_name].append(var_value)
                else:
                    pred_answers[var_name] = [var_value]
    if (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) == 0):
        total_acc += 1
        total_rec += 1
        total_pre += 1
        total_f1 += 1
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) > 0) and (len(list(pred_answers.keys())) == 0):
        print(f'\nempty predicted answer set for sample #{ids[i]}')
        continue
    elif (type(gold_ans[i]) != bool) and (len(gold_ans[i]) == 0) and (len(list(pred_answers.keys())) > 0):
        print(f'\npredicted answer set was supposed to be empty for sample #{ids[i]}')
        continue
    elif type(gold_ans[i]) == bool:
        print(f'\nresults are not of type boolean for sample #{ids[i]}')
        continue
    try:
        assert len(list(pred_answers.keys())) == 1
    except:
        print(f'\nresults dict has more keys than expected for sample #{ids[i]}')
        continue
    pred_ans = next(iter(pred_answers.values()))
    acc, rec, pre, f1 = eval(list(map(lambda x: process_text(x), gold_ans[i])), pred_ans)
    total_acc += acc
    total_rec += rec
    total_pre += pre
    total_f1 += f1
    # print(f'\ninfo for sample #{ids[i]}:')
    # print(f'acc = {acc}, rec = {rec}, pre = {pre}, f1 = {f1}')
    # print('\n'.join(gold_ans[i]))
    # print('---')
    # print('\n'.join(pred_ans))
    # print()
print(f'\nacc: {total_acc / len(queries)}')
print(f'rec: {total_rec / len(queries)}')
print(f'pre: {total_pre / len(queries)}')
print(f'f1: {total_f1 / len(queries)}')

  1%|▏         | 2/150 [00:01<01:42,  1.45it/s]


error executing SPARQL query: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error SR186:SECURITY: No permission to execute procedure DB.DBA.SPARUL_RUN with user ID 111, group ID 111\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n'

syntax error for sample #1


  2%|▏         | 3/150 [00:02<01:44,  1.41it/s]


results dict has more keys than expected for sample #2


  3%|▎         | 4/150 [00:02<01:41,  1.44it/s]

...


empty predicted answer set for sample #146


 99%|█████████▊| 148/150 [02:20<00:01,  1.42it/s]


error executing SPARQL query: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error SR186:SECURITY: No permission to execute procedure DB.DBA.SPARUL_RUN with user ID 111, group ID 111\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n'

syntax error for sample #147


100%|██████████| 150/150 [02:22<00:00,  1.05it/s]


error executing SPARQL query: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error SR186:SECURITY: No permission to execute procedure DB.DBA.SPARUL_RUN with user ID 111, group ID 111\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n'

syntax error for sample #149

acc: 0.4090507719442305
rec: 0.4375458116164819
pre: 0.4090507719442305
f1: 0.4086512859378727


#### ChatGPT API

In [ ]:
import json
import os
import openai
from tqdm import trange

def preprocess_text(text):
  return text

def lists_to_string(input_lists, limit=10, delim='\n'):
  # depth=3
  output_string = ''
  i = 0
  for x in range(len(input_lists)):
    for y in range(len(input_lists[x])):
      if i >= limit:
        return output_string.strip()
      output_string += delim.join(input_lists[x][y]) + delim
      i += 1
  return output_string.strip()

def extract_sparql_rule_based(response, start="```sparql", end="```"):
    include = False
    sparql = ''
    for line in response.split('\n'):
        if line == start:
            include = True
            continue
        if include:
            if line == end:
                break
            else:
                sparql += line + '\n'
    return sparql[:-1]

openai.api_key = "secret"

generated_responses = []
data = []
with open('/content/t5-3b-tri-10-ep-10-t_qald-9-plus_v3.json', 'r') as f:
  data = json.load(f)
  # queries = list(map(lambda x: preprocess_text(x['question']) + '\n' + preprocess_text(lists_to_string(x['new_triples'])), data))

for i in trange(len(data)):
    test_sample = data[i]

    first_response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Your task is to correct the syntax of the given DBPedia SPARQL. Do not generate anything more than the SPARQL. Begin the code with ```sparql and end it with ```. If unspecified, consider the LIMIT equal to 100."
                # "content": "Your task is to generate the DBPedia SPARQL of the given question w.r.t. the following relevant triples. Do not generate anything more than the SPARQL. Begin the code with ```sparql and end it with ```. If unspecified, consider the LIMIT equal to 100."
            },
            {
                "role": "user",
                # "content": queries[i]
                "content": test_sample['pred']
                # "content": test_sample['question'],
            },
        ],
        temperature=0.2,
        max_tokens=200,
        top_p=0.1,
    ).choices[0].message.content.strip()

    # second_response = openai.chat.completions.create(
    #     model="gpt-3.5-turbo",
    #     messages=[
    #         {
    #             "role": "system",
    #             "content": "Your task is to correct the syntax of the given DBPedia SPARQL. Do not generate anything more than the SPARQL. Begin the code with ```sparql and end it with ```. If unspecified, consider the LIMIT equal to 100."
    #             # "content": "Your task is to generate the DBPedia SPARQL of the given question. Do not generate anything more than the SPARQL. Begin the code with ```sparql and end it with ```. If unspecified, consider the LIMIT equal to 100."
    #         },
    #         {
    #             "role": "user",
    #             "content": test_sample['pred']
    #             # "content": test_sample['question'],
    #         },
    #         {
    #             "role": "assistant",
    #             "content": first_response,
    #         },
    #         {
    #             "role": "user",
    #             "content": "It still has syntax errors! Try again.",
    #             # "content": "It did not work! Try again.",
    #         }
    #     ],
    #     temperature=0.2,
    #     max_tokens=200,
    #     top_p=0.1,
    # ).choices[0].message.content.strip()

    generated_responses.append({
        "id": i,
        "gold_answers": test_sample['ans'],
        # "gold_answers": test_sample['new_answers'],
        "pred_sparql": extract_sparql_rule_based(first_response),
        # "pred_sparql_2": extract_sparql_rule_based(second_response)
    })

with open("gpt-3.5-turbo_t5-3b-tri-10-ep-10-t_qald-9-plus_v3.json", "w") as json_file:
    json.dump(generated_responses, json_file, indent=4)